## 06.1 Web Scraping Geospatial Data

In this notebook, we will examine how we can scrape metadata from the web to transform it into geospatial data. The web contains all kinds of geospatial data, if you know where to look for it. For example, headers on Twitter/X posts sometimes contain the geographic location from which they were sent (you can and should turn this feature off, but it's useful), photos sometimes have information from the GPS chip in the camera or phone where the picture was taken (again, you can turn this off if you share your photos), and your Strava posts certainly can be mined for geographic data (if you're interested in how this can be __very__ bad for OPSEC in "certain careers", see [this article](https://www.bellingcat.com/resources/articles/2018/07/08/strava-polar-revealing-homes-soldiers-spies/)).

Here we won't be doing anything nefarious, but will be addressing a minor annoyance in the serving of lidar point cloud data. For the past decade or so, the USGS has sought to perform lidar surveys over the entire US to provide updated topographic information via the 3D Elevation Program (3dep, [https://www.usgs.gov/3d-elevation-program](https://www.usgs.gov/3d-elevation-program)). Individual surveys are performed in support of a variety of agencies, and the data processed and served by the USGS in the form of derived data products like gridded digital elevation models (DEMs) and canopy height models (CHMs). However, they also preserve the raw point cloud data as compressed las files (along with metadata for those files), which are potentially useful for analyzing structural characteristics of forests in the UBRB. However, they aren't served in a particularly helpful format. Here is a link to the raw project data: [https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/ID_FEMAHQ_2018_D18/ID_FEMAHQ_2018/](https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/ID_FEMAHQ_2018_D18/ID_FEMAHQ_2018/). 

In the metadata folder are something like 10,000 XML files that contain metadata on each of the approximately 1 km${}^2$ point cloud tiles. In the LAZ folder, meanwhile, is the link to the LAS data for each of those tiles. The problem? We don't have a way of knowing where each of those LAS files is located on earth, except to click through the XML file and look at the bounds of each tile. Instead, we'll use some web sleuthing and scraping to read every one of those XML files, take note of the tile ID and the las file download URL, and we'll assemble a geospatial data file (a GeoDataFrame, specifically) that we can then examine on an interactive map. This will allow us to zoom around the UBRB and identify the specific tiles that we need and download them directly. We could also use our trusty GeoPandas skills to identify only those tiles that fall within a study area (like a watershed), use the `.clip()` operator to get that information, and then export the table of download URLs for LAS files in our study area. Below we do the first and most time-consuming step: reading all of the XML files, getting the bounding coordinates of each lidar tile, and assembling them into a GeoDataFrame.

### 1. Imports and Definitions

In [ ]:
import requests # Needed to open a web site
from bs4 import BeautifulSoup # Needed to parse an HTML website
from pyproj import CRS # Needed to define thee CRS of our output data file
from shapely.geometry import box # Needed to create features for our geodataframe
import geopandas as gpd # Needed to create our geospatial data layer

# Define a potential list of projections for the project lidar data
PROJ_NAME_MAP = {
    'NAD83 / Conus Albers': ('aea', 'NAD83'),
    'Albers Conical Equal Area': ('aea', None),
    'Lambert Conformal Conic': ('lcc', None),
    'Transverse Mercator': ('tmerc', None),
    # extend as you encounter new projection names
}

# Base URL for the project – this could be changed for other projects
gs_3dep_url = 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/ID_FEMAHQ_2018_D18/ID_FEMAHQ_2018/'

# XML Metadata files would be here
gs_3dep_meta_url = 'metadata/'

# JPG browse images would be here
gs_3dep_browse_url = 'browse/'

# Raw LAS files would be here
gs_3dep_las_url = 'LAZ/'

output_geojson_geo = 'ID_FEMAHQ_2018_las_tiles.geojson' # Output geospatial data layer in geographic projection
output_geojson_proj = 'ID_FEMAHQ_2018_las_tiles_alb.geojson' # Output geospatial data layer in albers projection

### 2. Function Definitions:

This first function will take the input URL where all of the XML metadata files can be found and return a list of links to those XML files.  

In [ ]:
def parse_html(gs_3dep_url):
    
    reqs = requests.get(gs_3dep_url) # Open the URL
    soup = BeautifulSoup(reqs.text, 'html.parser') # Create a structure to parse

    xml_links = [] # Empty list to which we will append XML file links
    for link_tag in soup.find_all('a', href=True): # Find all <a> tags with an href attribute
        href = link_tag.get('href')
        if href and href.lower().endswith('.xml'): # Check if the link ends with '.xml'
            # You may need to construct absolute URLs if hrefs are relative
            if href.startswith('http') or href.startswith('https'):
                xml_links.append(href)
            else:
                # Handle relative URLs (basic example, might need more robust handling for complex cases)
                from urllib.parse import urljoin
                absolute_url = urljoin(gs_3dep_url, href)
                xml_links.append(absolute_url)
                
    return xml_links

This second function take an individual URL to an XML metadata file for a 1 km${}^2$ lidar tile, opens the XML file, and parses that XML file to get the bounding coordinates of that lidar file. It also finds projection information from the metadata file. It returns a dictionary containing the bounding box of the lidar tile, and the CRS of the tile.

In [3]:
def parse_tile_xml(url: str) -> dict:
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, 'xml')

    bounds_xml = soup.find('spdom').find('bounding')
    bbox = {
        'west': float(bounds_xml.find('westbc').text),
        'east': float(bounds_xml.find('eastbc').text),
        'north': float(bounds_xml.find('northbc').text),
        'south': float(bounds_xml.find('southbc').text),
    }

    mapproj = soup.find('horizsys').find('planar').find('mapproj')
    proj_name = mapproj.find('mapprojn').text.strip()

    # prefer EPSG or WKT if present
    epsg_tag = soup.find('refsysid')
    if epsg_tag and epsg_tag.find('code'):
        crs = CRS.from_epsg(int(epsg_tag.find('code').text))
    elif mapproj.find('planarco'):
        crs = CRS.from_wkt(mapproj.find('planarco').text)
    else:
        proj_alias, datum = PROJ_NAME_MAP.get(proj_name, (None, None))
        params = {}
        if proj_alias == 'aea' and mapproj.albers:
            stdpars = mapproj.albers.find_all('stdparll')
            params = {
                'proj': 'aea',
                'lat_1': float(stdpars[0].text),
                'lat_2': float(stdpars[1].text),
                'lat_0': float(mapproj.albers.find('latprjo').text),
                'lon_0': float(mapproj.albers.find('longcm').text),
                'x_0': float(mapproj.albers.find('feast').text),
                'y_0': float(mapproj.albers.find('fnorth').text),
                'datum': datum or 'WGS84',
                'units': 'm',
            }
        # add elif blocks for other projections (lcc, tmerc, utm...) using their tags
        if params:
            crs = CRS.from_dict(params)
        else:
            crs = CRS.from_user_input(proj_name)  # final fallback

    return {'bbox': bbox, 'crs': crs}

### 3. Create the Geospatial Data Layer

Now we will call the functions above to create our data layer. First, parse the metadata HTML site to get links to the individual XML files.

In [ ]:
# Get the URLs for all of the XML files
project_xmls = parse_html(gs_3dep_url+gs_3dep_meta_url) 

Now, loop through that list of XML files and parse them for bounding boxes. Append these to a list that we will use to create a GeoDataFrame.

In [ ]:
# Get the metadata for the first XML file, we'll use this to set the 
# CRS for all tiles in this project
project_meta = parse_tile_xml(project_xmls[0])  
project_crs = project_meta['crs']

# Create an empty list to which we'll append the information for each XML file/lidar tile
records = []
for i, url in enumerate(project_xmls): # enumerate gives us both the individual url and an index
    meta = parse_tile_xml(url) # Get the bounding box for this XML file
    records.append(
        {
            'tile_id': url[-14:-4], # ID of the tile in something like wXXXXnXXXX format
            'xml_url': url, # Link to the xml file
            'browse_url': url.replace(gs_3dep_meta_url,gs_3dep_browse_url)[0:-4]+'.jpg', # Link to the JPG browse image
            'laz_url': url.replace(gs_3dep_meta_url,gs_3dep_las_url)[0:-4]+'.laz', # Link to the raw, compressed las file
            'geometry': box(meta['bbox']['west'], meta['bbox']['south'],
                            meta['bbox']['east'], meta['bbox']['north']), # Geometry of this feature
        }
    )
    
    # Print progress
    print(f'Completed {i/len(project_xmls):.2%} of XML files...', end='\r', flush=True)

# Create the GeoDataFrame from the XML records
tiles_gdf_geo = gpd.GeoDataFrame(records, crs='EPSG:4326')

# Create another version in the projection of the lidar files themselves
tiles_gdf_proj = tiles_gdf_geo.to_crs(project_crs)

### 4. Save Output

In [6]:
tiles_gdf_geo.to_file(output_geojson_geo)
tiles_gdf_proj.to_file(output_geojson_proj)
